## Setup

In [6]:
import os
import csv
from tqdm.notebook import tqdm
import numpy as np
import math
import torch
from IPython import display
import subprocess
import VARIABLES
import importlib
importlib.reload(VARIABLES)
from VARIABLES import *


In [2]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.47  Python-3.12.3 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3050, 8192MiB)
Setup complete  (32 CPUs, 31.8 GB RAM, 552.8/930.7 GB disk)


In [3]:
!pip install supervision

from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.19.0


In [8]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [0, 1]

Model summary (fused): 268 layers, 68125494 parameters, 0 gradients


## Utils

In [8]:
import VARIABLES
import importlib
importlib.reload(VARIABLES)
from VARIABLES import *

import utils
from utils import *
importlib.reload(utils)
from utils import *

videos_folder = "probando2"
videos_folder_path = os.path.join("full_recordings", videos_folder)


results_path = os.path.join("processing_results", videos_folder)
counts_path = os.path.join(results_path, "counts")
videos_output_path = os.path.join(results_path, "videos_output")


os.makedirs(results_path, exist_ok=True)
os.makedirs(counts_path, exist_ok=True)
os.makedirs(videos_output_path, exist_ok=True)


max_counters = {
  class_: LineZoneMaxCounterHelper(class_, CLASS_NAMES_DICT[class_]) 
  for class_ in selected_classes
}


vih = VideoInfoHandler()

for file_name in tqdm(os.listdir(videos_folder_path), desc=" Videos remaining", position=0):
  file_path = os.path.join(videos_folder_path, file_name)
  vih.re_init(file_path)
  process_video(
    source_path = file_path,
    target_path = os.path.join(videos_output_path, file_name),
    model = model,
    selected_classes = selected_classes,
    callback = callback,
    vih = vih
  )
  
  for class_ in selected_classes:
    max_counters[class_].update_counting(vih.line_zones, file_name)


save_results(max_counters, selected_classes, counts_path)


 Videos remaining:   0%|          | 0/2 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/365 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/365 [00:00<?, ?it/s]